## Find Cross-selling opportunities - Market Basket Analysis
#### <b> 문제인식 </b>
- 코로나 이후 신규 모집된 고객의 Retention 차원에서 적절한 Next 제품을 제안하는 것이 중요해짐
- 악세서리의 매출 증대 위해 JEW/ACC 기존고객에게 ACC제품으로의 cross-selling 가능성 모색 >> <b> 카테고리별 cross-selling 패턴 확인할것!! </b>

#### <b>고려사항</b>
2. 고객단위 vs 영수증단위 : Two track으로 시도할것
3. 제품별 categorization - proper level of categorization may be needed
- 반지(JEW-CAT) groupbby : 커플링, 웨딩링 구매를 cross-sell로 보기 어려울 것 같고, cross-category를 cross-sell로 규정해야할듯 데이터로 어떻게 조작?
5. ecom 고객과 매장 고객 cross-sell차이 (고객 프로파일링이 다를것으로 예상됨. 구분하여 분석)
5. time-line 구분 가능 => 
6. Virtual Items(요일, 시간대, 계절, 지역, 성별, 연령대) If shopper in 20's and night then cup ramen / Membership ID(=>울회사 by VIP level) could show other behavior

In [1]:
import numpy as np
import pandas as pd

In [2]:
# !pip install plotly chart_studio --upgrade
# !pip install cufflinks --upgrade

import chart_studio.plotly as py #chart_studio 는 plotly 로 작성된 그래프를 웹상에서 보여주기 위한 라이브러리
import cufflinks as cf #pandas로 그래프를 그리기 위한 기능을 plotly 와 연결시키기 위한 라이브러리
cf.go_offline(connected=True)

In [3]:
import matplotlib as mpl
from matplotlib import pyplot as plt

plt.rcParams['font.family'] = 'Gulim'
mpl.rcParams['axes.unicode_minus'] = False # 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처

In [4]:
import warnings # 경고 알림 제거
warnings.filterwarnings("ignore", category=Warning) # 경고 알림이 뜨면 모두 무시합니다.

pd.set_option('display.max_columns', 3000)
# pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x) #.0f: 소수점 자리수 지정

### 1) sales & customer data 전처리

#### 1-1) customer data를 중심으로 합침
- 2017년도에 코스모 리턴방식이 변경되었음(2017년도에는 동일 제품-시리얼-의 경우 날짜가 달라도 같은 영수증번호였음) > 이후 bw와 동일하게 시간,날짜 다를때 다른 영수증번호
- 따라서 2017년은 bw와 미맵핑된 것이 많음. 2017년 살리기 위해 cosmo 사용
- 단, 2017년 10월/11월음 중복값이 많아 중복값 제거하여, 수량 1개씩만 찍히는 코스모데이터에서 수량 2개 이상 팔린것(약 1억 천)이 같이 지워짐(장바구니 분석과 코호트 분석에서는 마이너 이슈라 pass)

In [5]:
%%time
# cosmo data에 매장으로 들어간 직원구매 제외시키기 위해 필요
df = pd.read_excel("DOS_Sales_2021.11.17.xlsx", sheet_name="ZO (2)") 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6170 entries, 0 to 6169
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Sales channel(key)             6170 non-null   object 
 1   Sales channel                  6170 non-null   object 
 2   TIME-Year                      6170 non-null   int64  
 3   TIME-Month                     6170 non-null   int64  
 4   TIME-Date                      6170 non-null   object 
 5   SHOP-Sales Time                6170 non-null   object 
 6   PC_Profit Center(key)          6170 non-null   object 
 7   PC_Profit Center               6170 non-null   object 
 8   CUST-Country/Region            6170 non-null   object 
 9   SHOP-Sales Person(key)         6170 non-null   object 
 10  SHOP-Sales Person              6170 non-null   object 
 11  SHOP-HRVision Code             6170 non-null   object 
 12  MAT-Classification(key)        5441 non-null   o

In [6]:
df = df[df['TIME-Year'] != 2016 ] # CRM 데이터 수집 기간과 맞춰줌 (2021.10.31일 기준)
df = df[df['MAT_Business Code(key)'].isin(['CAR', '125', 'BAS', '111', '112', 'GFT', '220', '240', 'WAT', 'LTR', 'SLG', 'SLK', 'HOD', 'EYE', 'SRM'])] #JWA sales only
df = df[df['PC_Profit Center(key)'] == '0450/224'] # 직원구매
df.shape

(4224, 29)

In [7]:
%%time
# 고객 데이터
df1 = pd.read_excel("DOS_Sales_2021.11.17.xlsx", sheet_name="POWER BI Raw data (Local)", header=10, usecols='B:AD') 
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291748 entries, 0 to 291747
Data columns (total 29 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   Main Store                        291395 non-null  object 
 1   Client Category                   290866 non-null  object 
 2   Purchase: Shop                    291748 non-null  object 
 3   COSMO ID                          291748 non-null  object 
 4   CRM ID                            291551 non-null  float64
 5   Gender                            288004 non-null  object 
 6   Last and First Name (Local)       291629 non-null  object 
 7   One Shot                          291748 non-null  bool   
 8   Birthdate Year                    105579 non-null  float64
 9   Segment                           277851 non-null  object 
 10  Main Client Advisor               290381 non-null  object 
 11  Nationality                       222804 non-null  o

In [8]:
# Text 타입(Object)를 날짜 형식(Datetime)으로 변환
df1['First Purchase Date'] = pd.to_datetime(df1['First Purchase Date'], yearfirst=True, errors='coerce')
df1['Purchase Date'] = pd.to_datetime(df1['Purchase Date'], yearfirst=True, errors='coerce')
df1['TIME-Year'] = df1['Purchase Date'].dt.year
df1['TIME-Month'] = df1['Purchase Date'].dt.month
df1.head()

,Main Store,Client Category,Purchase: Shop,COSMO ID,CRM ID,Gender,Last and First Name (Local),One Shot,Birthdate Year,Segment,Main Client Advisor,Nationality,Marketing Authorization,First Purchase Date,Last Purchase Date General,Transaction Type Details,Reference Transaction Beanstore2,SBU,Product Family,Aesthetic Motive,Aesthetic Line,Material Number (SAP Code),Catalog Code,Product Name,Purchase Date,Item Quantity,Discount %,Sales Amount (Purchase Currency),Lifetime Value (User Currency),TIME-Year,TIME-Month
0,Starfield hanam,Person,Starfield hanam,0015800000w4QoOAAU,7135003.000,Male,오승묵,False,1984.000,Dormant,Jina Yun(21765),NaN,True,2017-01-01,2017. 1. 1,Retail-Return mchdse,053702000001258,JEW,Rings-F-C,B.zero1,B.zero1,335937,AN852405,AN B01 PINK GOLD 3 BANDS S.56,2017-01-01,1.000,NaN,-2350000.000,4500000.000,2017,1
1,Starfield hanam,Person,Starfield hanam,0015800000w4QoOAAU,7135003.000,Male,오승묵,False,1984.000,Dormant,Jina Yun(21765),NaN,True,2017-01-01,2017. 1. 1,Retail-Mchdise sale,053702000001258,JEW,Rings-F-C,B.zero1,B.zero1,335937,AN852405,AN B01 PINK GOLD 3 BANDS S.56,2017-01-01,1.000,NaN,2350000.000,4500000.000,2017,1
2,Starfield hanam,Person,Starfield hanam,0015800000w4QoOAAU,7135003.000,Male,오승묵,False,1984.000,Dormant,Jina Yun(21765),NaN,True,2017-01-01,2017. 1. 1,Retail-Mchdise sale,053702000001260,JEW,Rings-F-C,Bvlgari Bvlgari,Bvlgari Bvlgari Bands,341827,AN854185,BAND RING BULGARI BULGARI PG DIA S.56,2017-01-01,1.000,NaN,2250000.000,4500000.000,2017,1
3,Starfield hanam,Person,Starfield hanam,0015800000w4QoOAAU,7135003.000,Male,오승묵,False,1984.000,Dormant,Jina Yun(21765),NaN,True,2017-01-01,2017. 1. 1,Retail-Mchdise sale,053702000001260,JEW,Rings-F-C,Bvlgari Bvlgari,Bvlgari Bvlgari Bands,341820,AN854185,BAND RING BULGARI BULGARI PG DIA S.49,2017-01-01,1.000,NaN,2250000.000,4500000.000,2017,1
4,Starfield hanam,Person,Starfield hanam,0015800000w4QoOAAU,7135003.000,Male,오승묵,False,1984.000,Dormant,Jina Yun(21765),NaN,True,2017-01-01,2017. 1. 1,Retail-Return mchdse,053702000001258,JEW,Rings-F-C,Bvlgari Bvlgari,Bvlgari Bvlgari Bands,341820,AN854185,BAND RING BULGARI BULGARI PG DIA S.49,2017-01-01,1.000,NaN,-2250000.000,4500000.000,2017,1


In [9]:
df1.shape

(291748, 31)

In [10]:
df1 = df1[~df1['SBU'].isin(['JAS', 'PER', 'WAS', 'AAS'])] # JWA sales only
df1 = df1[~df1['Client Category'].isin(['Employee'])] # w.o employee sales (family sales는 purchase shop에서 ofc로 표기되므로 매장에서 걸러짐)
df1 = df1[(df1['Transaction Type Details']=="Retail-Mchdise sale")|(df1['Transaction Type Details']=="Retail-Return mchdse")] #제품 영수증만(기프트 영수증 및 디파짓 제외)
df1 = df1[~df1['Sales Amount (Purchase Currency)'].isnull()]
df1.shape

(182149, 31)

In [11]:
StringList = df1['Purchase: Shop'].tolist()
set([p for p in StringList if not(p.startswith(('Bvlgari', 'BVLGARI', 'Bulgari')))]) #우리매장만 골라내기

{'AK Plaza Bundang',
 'BANGKOK EMPORIUM',
 'BRISBANE',
 'Bangkok Siam Paragon',
 'CHENGDU IFS',
 'CWTC Store, Beijing',
 'Central Phuket Floresta',
 'Deji store, Nanjing',
 'E-Commerce FR',
 'E-Commerce KR',
 'Galleria Gwanggyo',
 'Galleria Luxury Hall (East)',
 'Gold Coast Pacific Fair Store',
 'GreenBelt 4',
 'Hyundai Apgujeong',
 'Hyundai Department Store Mokdong',
 'Lotte Avenuel',
 'Lotte Avenuel World Tower',
 'Lotte Department Store Busan Avenuel',
 'Lotte Department Store Centumcity',
 'Lotte Department Store Daegu',
 'Lotte Department Store Gwangju',
 'Lotte Department Ulsan',
 'MACAU GALAXY',
 'MIXC Store, Shenzhen',
 'Macau Wynn Palace',
 'OFC',
 'Pangyo Hyundai',
 'Paris Galeries Lafayette',
 'Plaza 66 Store, Shanghai',
 'Seoul Galleria West',
 'Seoul Hyundai Parc1',
 'Shinsegae Daegu',
 'Shinsegae Deparment Store Gangnam',
 'Shinsegae Department Store Centum City',
 'Shinsegae Department Store Gyeonggi',
 'Shinsegae Department Store Yeongdeungpo',
 'Starfield hanam',
 'Tai

In [12]:
shop = ["AK Plaza Bundang","E-Commerce KR","Galleria Gwanggyo","Galleria Luxury Hall (East)","Hyundai Apgujeong",\
       "Hyundai Department Store Mokdong","Lotte Avenuel","Lotte Avenuel World Tower","Lotte Department Store Busan Avenuel",\
       "Lotte Department Store Centumcity","Lotte Department Store Daegu","Lotte Department Store Gwangju","Lotte Department Ulsan",\
       "Pangyo Hyundai",'Seoul Galleria West', 'Seoul Hyundai Parc1', "Shinsegae Daegu","Shinsegae Deparment Store Gangnam","Shinsegae Department Store Centum City",\
       "Shinsegae Department Store Gyeonggi","Shinsegae Department Store Yeongdeungpo","Starfield hanam"]
len(shop)

22

In [13]:
df1 = df1[df1['Purchase: Shop'].isin(shop)]
df1.shape

(180966, 31)

In [14]:
df1['Sales Amount (Purchase Currency)'].sum()

621775497183.0

In [15]:
#bi bw raw data pc 224 sales invoice를 코스모 데이터에서 제외 (별로없음)
emp = df['Sales ID BS'].unique().tolist()
print(df1[df1['Reference Transaction Beanstore2'].isin(emp)]['Sales Amount (Purchase Currency)'].sum() / 1.1)
df1 = df1[~df1['Reference Transaction Beanstore2'].isin(emp)]
df1.shape

234449999.99999997


(180825, 31)

In [16]:
df1['Sales Amount (Purchase Currency)'].sum() / 1.1

565016001984.5454

In [17]:
df1.reset_index(drop=True, inplace=True)
print(df1.shape)
df1.info()

(180825, 31)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180825 entries, 0 to 180824
Data columns (total 31 columns):
 #   Column                            Non-Null Count   Dtype         
---  ------                            --------------   -----         
 0   Main Store                        180500 non-null  object        
 1   Client Category                   180821 non-null  object        
 2   Purchase: Shop                    180825 non-null  object        
 3   COSMO ID                          180825 non-null  object        
 4   CRM ID                            180685 non-null  float64       
 5   Gender                            178843 non-null  object        
 6   Last and First Name (Local)       180754 non-null  object        
 7   One Shot                          180825 non-null  bool          
 8   Birthdate Year                    66632 non-null   float64       
 9   Segment                           173479 non-null  object        
 10  Main Client Advisor

In [18]:
cosmo = []
for i in range(2017, 2022):
    dfshop = df1[(df1['TIME-Year']==i)]['Sales Amount (Purchase Currency)'].sum()/1.1
    cosmo.append(dfshop)
cosmo = pd.DataFrame(cosmo, columns=['cosmo'])
cosmo

,cosmo
0,64902216590.909
1,73758119800.000
2,89986545325.455
3,123964575572.727
4,212404544695.455


In [19]:
md = [64785888018, 74203771726, 90169871712, 123572022972, 215761181700] #b2와 매칭되는 정도 파악
md = pd.DataFrame(md, columns=['bi'])
df2 = pd.concat([cosmo, md], axis=1) #bw, 
df2['cosmo/bi'] = df2['cosmo'] / df2['bi']
df2['cosmo-bi'] = df2['cosmo'] - df2['bi']
# df2['bw/bi'] = df2['bw'] / df2['bi']
# df2['bw-bi'] = df2['bw'] - df2['bi']
df2

,cosmo,bi,cosmo/bi,cosmo-bi
0,64902216590.909,64785888018,1.002,116328572.909
1,73758119800.000,74203771726,0.994,-445651926.000
2,89986545325.455,90169871712,0.998,-183326386.545
3,123964575572.727,123572022972,1.003,392552600.727
4,212404544695.455,215761181700,0.984,-3356637004.545


In [20]:
# 괴리가 클시 확인해볼것
# df1[df1['Purchase Date'].dt.year == 2021].set_index('Purchase Date').groupby(pd.Grouper(freq='M'))['Sales Amount (Purchase Currency)'].sum() / 1.1

In [21]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180825 entries, 0 to 180824
Data columns (total 31 columns):
 #   Column                            Non-Null Count   Dtype         
---  ------                            --------------   -----         
 0   Main Store                        180500 non-null  object        
 1   Client Category                   180821 non-null  object        
 2   Purchase: Shop                    180825 non-null  object        
 3   COSMO ID                          180825 non-null  object        
 4   CRM ID                            180685 non-null  float64       
 5   Gender                            178843 non-null  object        
 6   Last and First Name (Local)       180754 non-null  object        
 7   One Shot                          180825 non-null  bool          
 8   Birthdate Year                    66632 non-null   float64       
 9   Segment                           173479 non-null  object        
 10  Main Client Advisor             

In [22]:
# df2: 고객데이터 필요한컬럼만 선택
df2 = df1.loc[:, ['TIME-Year', 'TIME-Month', 'Purchase Date', 'Purchase: Shop', 'Main Client Advisor', 'COSMO ID', 'Transaction Type Details', 'Reference Transaction Beanstore2', \
                  'Material Number (SAP Code)', 'Item Quantity', 'Discount %', 'Sales Amount (Purchase Currency)']]\
.rename(columns={'Purchase Date':'TIME-Date', 'Reference Transaction Beanstore2': 'Sales ID BS', 'Material Number (SAP Code)':'MAT_Material(key)', 'Item Quantity':'UNIT SOLD', \
                 'Sales Amount (Purchase Currency)':'NET VALUE'}) #부가세 포함 금액임 (코드짜놓은거 재사용하기위해바꿈)
print(len(df2))
df2.head()

180825


,TIME-Year,TIME-Month,TIME-Date,Purchase: Shop,Main Client Advisor,COSMO ID,Transaction Type Details,Sales ID BS,MAT_Material(key),UNIT SOLD,Discount %,NET VALUE
0,2017,1,2017-01-01,Starfield hanam,Jina Yun(21765),0015800000w4QoOAAU,Retail-Return mchdse,053702000001258,335937,1.000,NaN,-2350000.000
1,2017,1,2017-01-01,Starfield hanam,Jina Yun(21765),0015800000w4QoOAAU,Retail-Mchdise sale,053702000001258,335937,1.000,NaN,2350000.000
2,2017,1,2017-01-01,Starfield hanam,Jina Yun(21765),0015800000w4QoOAAU,Retail-Mchdise sale,053702000001260,341827,1.000,NaN,2250000.000
3,2017,1,2017-01-01,Starfield hanam,Jina Yun(21765),0015800000w4QoOAAU,Retail-Mchdise sale,053702000001260,341820,1.000,NaN,2250000.000
4,2017,1,2017-01-01,Starfield hanam,Jina Yun(21765),0015800000w4QoOAAU,Retail-Return mchdse,053702000001258,341820,1.000,NaN,-2250000.000


In [23]:
print(df2['COSMO ID'].nunique())
# df3: 순수 고객데이터베이스
df3 = df1[['COSMO ID', 'Gender', 'Last and First Name (Local)', 'Birthdate Year', 'Segment', 'Nationality', 'First Purchase Date', 'Last Purchase Date General', 'Lifetime Value (User Currency)']] #CRM ID blank생겨서 사용안함
df3.drop_duplicates(subset=['COSMO ID'], keep='last', inplace=True, ignore_index=True)
df3.shape #df2와 고객수 같은지 확인

86854


(86854, 9)

In [24]:
# 고객데이터 병합
df4 = pd.merge(df2, df3, how='left', on = 'COSMO ID')
df4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180825 entries, 0 to 180824
Data columns (total 20 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   TIME-Year                       180825 non-null  int64         
 1   TIME-Month                      180825 non-null  int64         
 2   TIME-Date                       180825 non-null  datetime64[ns]
 3   Purchase: Shop                  180825 non-null  object        
 4   Main Client Advisor             180793 non-null  object        
 5   COSMO ID                        180825 non-null  object        
 6   Transaction Type Details        180825 non-null  object        
 7   Sales ID BS                     180825 non-null  object        
 8   MAT_Material(key)               180825 non-null  object        
 9   UNIT SOLD                       180825 non-null  float64       
 10  Discount %                      15964 non-null   float64

#### 1-2) 실제 고객 분석에서 제외할 데이터 추가 편집
1. **리턴 영수증 중 0 되는 것 삭제**
    - 1. df raw data 단계에서 bs id & mat 기준 net value sum 0인것 제외 
    - 2. cosmo id & mat 기준 groupby 한 후 net value sum 0 '미만'인거 제외
    - 3. cosmo id & mat & TIME-Date기준 : 같은날 리턴해서 0되는것 제거 
    - 5. 데이터 정렬 후, 리턴 영수증 위에행이랑 sales 합이 0이면 두 행 제거(리턴 하나전, 리턴 하나 뒤, 리턴 두개 뒤, 리턴 하나전 다시, 리턴 두개 전, 리턴 하나 뒤 다시)
2. 세일즈, 수량 마이너스 고객 제외
3. 고객 제외
    - **비회원, 단체구매(SK) 제외, 수량 제일 많은(118개) 다이고 의심고객 제외** 
    - **별도의 파일 저장없이 Transaction 기준 장바구니 분석시에는 이거 살리고, 고객기준 장바구니 분석시에는 3번 코드 실행하여 제외**

In [25]:
df4.groupby(['Transaction Type Details'])['Sales ID BS'].nunique()

Transaction Type Details
Retail-Mchdise sale     126421
Retail-Return mchdse      9801
Name: Sales ID BS, dtype: int64

##### 1. 영수증 제외
- df7: 티켓별 sum 0 
- df8: 고객별 sum < 0

In [26]:
df0 = df4.copy()
print(len(df0))
df0['NET VALUE'].sum() #삭제 전후 비교위해

180825


621517602183.0

In [27]:
df7 = df0.groupby(['Sales ID BS', 'MAT_Material(key)'])['NET VALUE'].sum().sort_values().reset_index()
df7 = df7[(df7['NET VALUE'] > -1) & (df7['NET VALUE'] < 1)] #net value sum 0(소수점때문에 범위로) 
print(len(df7))

df7['key0'] = df7['Sales ID BS'] + df7['MAT_Material(key)']
ticket7 = list(df7['key0'])
print(len(ticket7))
df0['key0'] = df0['Sales ID BS'] + df0['MAT_Material(key)']
num0 = df0[df0['key0'].isin(ticket7)]['NET VALUE'].sum()
f"{num0:.2f}" #0인지 확인하고 ticket7 제외할것

2705
2705


'0.00'

In [28]:
df0 = df0[~df0['key0'].isin(ticket7)]
print(len(df0)) #영수증 제외 전후 비교
df0['NET VALUE'].sum()

175371


621517602183.0

In [29]:
df8 = df0.groupby(['COSMO ID', 'MAT_Material(key)'])['NET VALUE'].sum().reset_index()
df8 = df8[(df8['NET VALUE'] <= 1)] #-1~1범위로 0 찾음, 2016년에 제품을 샀다가 리턴했으면 -세일즈 나옴 (보통 리턴 후 다른제품 구매) (df8['NET VALUE'] >= -1) & 
print(len(df8))
print(df8[(df8['NET VALUE'] <= 1)]['NET VALUE'].sum()) #제외되는 금액

df8['key'] = df8['COSMO ID'] + df8['MAT_Material(key)']
ticket8 = list(df8['key'])
df0['key'] = df0['COSMO ID'] + df0['MAT_Material(key)']
num = df0[df0['key'].isin(ticket8)]['NET VALUE'].sum() #최종 db에서 제외되는 금액 확인
print(f"{num:.2f}")

3815
-391333390.0
-391333390.00


In [30]:
df0 = df0[~df0['key'].isin(ticket8)]
print(len(df0)) #약 8천개 미만의 행을 추가 삭제
print(df0['NET VALUE'].sum()) #금액 전후 비교

167352
621908935573.0


In [31]:
df0 = df0[df0.groupby(['COSMO ID', 'TIME-Date', 'MAT_Material(key)'])["NET VALUE"].transform(sum) != 0] #같은날 리턴해서 0되는것 제거

In [32]:
print(len(df0))
print(df0['NET VALUE'].sum()) #리턴 편집전과 세일즈 토탈 금액 비교할것 

165514
621908935573.0


In [33]:
# df0이 바뀌었기 때문에 한번 또돌림
df7 = df0.groupby(['Sales ID BS', 'MAT_Material(key)'])['NET VALUE'].sum().sort_values().reset_index()
df7 = df7[(df7['NET VALUE'] > -1) & (df7['NET VALUE'] < 1)] #net value sum 0(소수점때문에 범위로) 
print(len(df7))

df7['key0'] = df7['Sales ID BS'] + df7['MAT_Material(key)']
ticket7 = list(df7['key0'])
print(len(ticket7))
df0['key0'] = df0['Sales ID BS'] + df0['MAT_Material(key)']
num0 = df0[df0['key0'].isin(ticket7)]['NET VALUE'].sum()
f"{num0:.2f}" #0인지 확인하고 ticket7 제외할것

2
2


'0.00'

In [34]:
df0 = df0[~df0['key0'].isin(ticket7)]
print(len(df0)) #영수증 아주 쪼금 제외 ㅎ
df0['NET VALUE'].sum() #전후 금액 비교

165510


621908935573.0

In [35]:
# a = pd.pivot_table(df0, index=['COSMO ID', 'MAT_Material(key)'], columns='Transaction Type Details', values='Sales ID BS', aggfunc='count')
# a[(a['Retail-Return mchdse']>=1)&(a['Retail-Mchdise sale']!=a['Retail-Return mchdse'])].sort_values(['Retail-Mchdise sale', 'Retail-Return mchdse']) #리턴이 포함되었으나 결국 세일즈 일으킨건, cf.같은 경우는 두건: 실제로 구매함
# print(len(b)) #다른날 리턴한건 별로안되어서(110건) 패스하자(3건중 두건을 지우는 방법 잘 모르겠음)

In [36]:
df0 = df0.sort_values(['COSMO ID', 'MAT_Material(key)', 'TIME-Date', 'Sales ID BS']).reset_index(drop=True) #리턴과 위에있는 세일즈 0될 경우 지우기 위해 정렬해줌

In [37]:
%%time
lst = []
pre = []
for index, row in df0.iterrows():
    if row['Transaction Type Details'] == 'Retail-Return mchdse': 
        if row['NET VALUE'] + df0.loc[index-1, 'NET VALUE'] == 0: #리턴 이전 행과 합이 0
            idx = index
            preidx = index-1
            lst.append(idx)
            pre.append(preidx)
result = lst + pre
result[:5]

Wall time: 15.5 s


[42, 45, 48, 51, 60]

In [38]:
df0[df0.index.isin(result)]['NET VALUE'].sum()

0.0

In [39]:
df0 = df0[~df0.index.isin(result)]
print(len(df0))
df0['NET VALUE'].sum() #야호 감동의 눈물 ㅠ.ㅠ원하는거 거의 근접하게 지움 ㅎㅎ

158330


621908935573.0

In [40]:
df0.groupby(['Transaction Type Details'])['Sales ID BS'].nunique() #리턴 편집전후로 여기서 가장 많이 줄어듬

Transaction Type Details
Retail-Mchdise sale     118609
Retail-Return mchdse       931
Name: Sales ID BS, dtype: int64

In [41]:
df0 = df0.reset_index(drop=True) #혹시 몰라 인덱스 모두 1차이 나도록 리셋

In [42]:
%%time
lst = []
aft = []
for index, row in df0.iterrows():
    if row['Transaction Type Details'] == 'Retail-Return mchdse':
        if row['NET VALUE'] + df0.loc[index+1, 'NET VALUE'] == 0: #리턴 이후 행과 합이 0
            idx = index
            aftidx = index+1
            lst.append(idx)
            aft.append(aftidx)
result = lst + aft
result[:5]

Wall time: 15.2 s


[652, 818, 1416, 1419, 1476]

In [43]:
df0[df0.index.isin(result)]['NET VALUE'].sum()

0.0

In [44]:
df0 = df0[~df0.index.isin(result)]
print(len(df0))
df0['NET VALUE'].sum() #야호 감동의 눈물 ㅠ.ㅠ원하는거 거의 근접하게 지움 ㅎㅎ

156362


621908935573.0

In [45]:
df0.groupby(['Transaction Type Details'])['Sales ID BS'].nunique() #리턴 편집전과 세일즈&리턴영수증 개수 비교

Transaction Type Details
Retail-Mchdise sale     117966
Retail-Return mchdse       113
Name: Sales ID BS, dtype: int64

In [46]:
df0 = df0.reset_index(drop=True) #혹시 몰라 인덱스 모두 1차이 나도록 리셋

In [47]:
%%time
lst = []
aft = []
for index, row in df0.iterrows():
    if row['Transaction Type Details'] == 'Retail-Return mchdse':
        if row['NET VALUE'] + df0.loc[index+2, 'NET VALUE'] == 0: #리턴 이후 다다음 행과 합이 0 (보통 가격인상전 디파짓걸어놓고 세일즈 클로징할때임)
            idx = index
            aftidx = index+2
            lst.append(idx)
            aft.append(aftidx)
result = lst + aft
result[:5]

Wall time: 15 s


[576, 2272, 4317, 6829, 9092]

In [48]:
df0[df0.index.isin(result)]['NET VALUE'].sum()

0.0

In [49]:
df0 = df0[~df0.index.isin(result)]
print(len(df0))
df0['NET VALUE'].sum() #세일즈는 계속 똑같은지 비교

156192


621908935573.0

In [50]:
df0.groupby(['Transaction Type Details'])['Sales ID BS'].nunique() #리턴 편집전과 세일즈&리턴영수증 개수 비교

Transaction Type Details
Retail-Mchdise sale     117918
Retail-Return mchdse        45
Name: Sales ID BS, dtype: int64

In [51]:
df0 = df0.reset_index(drop=True)

In [52]:
%%time
lst = []
pre = []
for index, row in df0.iterrows():
    if row['Transaction Type Details'] == 'Retail-Return mchdse': 
        if row['NET VALUE'] + df0.loc[index-1, 'NET VALUE'] == 0: #리턴 이전 행과 합이 0 다시한번
            idx = index
            preidx = index-1
            lst.append(idx)
            pre.append(preidx)
result = lst + pre
result[:5]

Wall time: 14.2 s


[10119, 15229, 19611, 35158, 35161]

In [53]:
df0[df0.index.isin(result)]['NET VALUE'].sum()

0.0

In [54]:
df0 = df0[~df0.index.isin(result)]
print(len(df0))
df0['NET VALUE'].sum() #세일즈는 계속 똑같은지 비교

156144


621908935573.0

In [55]:
df0.groupby(['Transaction Type Details'])['Sales ID BS'].nunique() #ㅋㅋ대박 다 지운듯

Transaction Type Details
Retail-Mchdise sale     117905
Retail-Return mchdse        26
Name: Sales ID BS, dtype: int64

In [56]:
df0 = df0.reset_index(drop=True)

In [57]:
%%time
lst = []
pre = []
for index, row in df0.iterrows():
    if row['Transaction Type Details'] == 'Retail-Return mchdse': 
        if row['NET VALUE'] + df0.loc[index-2, 'NET VALUE'] == 0: #리턴 전전 행과 합이 0
            idx = index
            preidx = index-2
            lst.append(idx)
            pre.append(preidx)
result = lst + pre
result[:5]

Wall time: 14.4 s


[11403, 25584, 28121, 29241, 37978]

In [58]:
df0[df0.index.isin(result)]['NET VALUE'].sum()

0.0

In [59]:
df0 = df0[~df0.index.isin(result)]
print(len(df0))
df0['NET VALUE'].sum() #세일즈는 계속 똑같은지 비교

156102


621908935573.0

In [60]:
df0.groupby(['Transaction Type Details'])['Sales ID BS'].nunique() #ㅋㅋ대박 다 지운듯

Transaction Type Details
Retail-Mchdise sale     117890
Retail-Return mchdse         7
Name: Sales ID BS, dtype: int64

In [61]:
df0 = df0.reset_index(drop=True) #혹시 몰라 인덱스 모두 1차이 나도록 리셋

In [62]:
%%time
lst = []
aft = []
for index, row in df0.iterrows():
    if row['Transaction Type Details'] == 'Retail-Return mchdse':
        if row['NET VALUE'] + df0.loc[index+1, 'NET VALUE'] == 0: #리턴 이후 행과 합이 0
            idx = index
            aftidx = index+1
            lst.append(idx)
            aft.append(aftidx)
result = lst + aft
result[:5]

Wall time: 13.7 s


[86866, 86869, 86867, 86870]

In [63]:
df0[df0.index.isin(result)]['NET VALUE'].sum()

0.0

In [64]:
df0 = df0[~df0.index.isin(result)]
print(len(df0))
df0['NET VALUE'].sum() #야호 감동의 눈물 ㅠ.ㅠ원하는거 거의 근접하게 지움 ㅎㅎ

156098


621908935573.0

In [65]:
df0.groupby(['Transaction Type Details'])['Sales ID BS'].nunique() #리턴 편집전과 세일즈&리턴영수증 개수 각각 6천개씩 줄임!

Transaction Type Details
Retail-Mchdise sale     117889
Retail-Return mchdse         6
Name: Sales ID BS, dtype: int64

In [66]:
df0[df0['Transaction Type Details']=='Retail-Return mchdse']

,TIME-Year,TIME-Month,TIME-Date,Purchase: Shop,Main Client Advisor,COSMO ID,Transaction Type Details,Sales ID BS,MAT_Material(key),UNIT SOLD,Discount %,NET VALUE,Gender,Last and First Name (Local),Birthdate Year,Segment,Nationality,First Purchase Date,Last Purchase Date General,Lifetime Value (User Currency),key0,key
3078,2019,3,2019-03-03,Shinsegae Department Store Gyeonggi,Younghyun Lee,0014H00001iO7qcQAC,Retail-Return mchdse,055801000015776,341820,1.000,NaN,-2450000.000,Female,황인선,1983.000,High Potential,Korean (KR)-South Korea,2019-03-03,2019. 3. 3,4840000.000,055801000015776341820,0014H00001iO7qcQAC341820
7911,2021,7,2021-07-03,Pangyo Hyundai,Lee Kiwoo,0014H00001krJiwQAE,Retail-Return mchdse,053602000009744,357573,1.000,0.030,-10476000.000,Male,장석우,NaN,Elite,Korean (KR)-South Korea,2019-05-25,2021. 7. 3,26480000.000,053602000009744357573,0014H00001krJiwQAE357573
83808,2021,11,2021-11-26,Shinsegae Deparment Store Gangnam,Munhyang Yoon,0014K00000QkkngQAB,Retail-Return mchdse,059802000066477,339981,1.000,NaN,-2860000.000,Male,김명진,1989.000,Premium,Korean (KR)-South Korea,2021-10-17,2021. 11. 26,5600000.000,059802000066477339981,0014K00000QkkngQAB339981
111118,2017,2,2017-02-11,Lotte Department Store Gwangju,JIAE PARK (602146),0015800000w4iaJAAQ,Retail-Return mchdse,057201000010688,351117,1.000,0.100,-3510000.000,Male,장주호,1965.000,Top,Korean (KR)-South Korea,2006-09-20,2021. 11. 20,408401387.390,057201000010688351117,0015800000w4iaJAAQ351117
115260,2017,1,2017-01-24,Hyundai Department Store Mokdong,Sunmin Lim(0000015065),0015800000wU1SyAAK,Retail-Return mchdse,057102000020273,349710,1.000,0.030,-5820000.000,Female,장홍자,NaN,Dormant,NaN,2006-10-12,2017. 1. 24,8067996.720,057102000020273349710,0015800000wU1SyAAK349710
127706,2020,9,2020-09-29,Pangyo Hyundai,Utenza TBR,0015800000z8vxWAAQ,Retail-Return mchdse,053601000008084,335992,1.000,NaN,-954000.000,Female,불가리고객,1979.000,Top,Korean (KR)-South Korea,2005-09-08,2011. 9. 30,3283541946.930,053601000008084335992,0015800000z8vxWAAQ335992


In [67]:
df0 = df0.drop(['key0','key'], axis=1) #컬럼 삭제
df0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 156098 entries, 0 to 156101
Data columns (total 20 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   TIME-Year                       156098 non-null  int64         
 1   TIME-Month                      156098 non-null  int64         
 2   TIME-Date                       156098 non-null  datetime64[ns]
 3   Purchase: Shop                  156098 non-null  object        
 4   Main Client Advisor             156070 non-null  object        
 5   COSMO ID                        156098 non-null  object        
 6   Transaction Type Details        156098 non-null  object        
 7   Sales ID BS                     156098 non-null  object        
 8   MAT_Material(key)               156098 non-null  object        
 9   UNIT SOLD                       156098 non-null  float64       
 10  Discount %                      12921 non-null   float64

In [68]:
#영수증 미맵핑 고객아이디 처리 >> groupby 인덱스칼럼에 null있으면 집계결과에 포함 안됨
df0.isnull().sum()

TIME-Year                              0
TIME-Month                             0
TIME-Date                              0
Purchase: Shop                         0
Main Client Advisor                   28
COSMO ID                               0
Transaction Type Details               0
Sales ID BS                            0
MAT_Material(key)                      0
UNIT SOLD                              0
Discount %                        143177
NET VALUE                              0
Gender                              1677
Last and First Name (Local)           55
Birthdate Year                     99092
Segment                             6272
Nationality                        31593
First Purchase Date                  438
Last Purchase Date General           464
Lifetime Value (User Currency)         0
dtype: int64

In [69]:
#영수증 미맵핑 고객아이디 처리 >> groupby 인덱스칼럼에 null있으면 집계결과에 포함 안됨
df0['COSMO ID'] = df0['COSMO ID'].fillna('Not assigned') 
df0.loc[df0[df0['Gender'].isnull()].index, 'Gender'] = 'Not assigned'
df0.loc[df0[df0['Last and First Name (Local)'].isnull()].index, 'Last and First Name (Local)'] = 'Not assigned'
df0.loc[df0[df0['Birthdate Year'].isnull()].index, 'Birthdate Year'] = 'Not assigned'
df0.loc[df0[df0['Segment'].isnull()].index, 'Segment'] = 'Not assigned'
df0.loc[df0[df0['Nationality'].isnull()].index, 'Nationality'] = 'Not assigned'
df0.loc[df0[df0['First Purchase Date'].isnull()].index, 'First Purchase Date'] = 'Not assigned'
df0.loc[df0[df0['Last Purchase Date General'].isnull()].index, 'Last Purchase Date General'] = 'Not assigned'
# df0.loc[df0[df0['Lifetime Value (User Currency)'].isnull()].index, 'Lifetime Value (User Currency)'] = 'Not Assigned'

In [70]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 156098 entries, 0 to 156101
Data columns (total 20 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   TIME-Year                       156098 non-null  int64         
 1   TIME-Month                      156098 non-null  int64         
 2   TIME-Date                       156098 non-null  datetime64[ns]
 3   Purchase: Shop                  156098 non-null  object        
 4   Main Client Advisor             156070 non-null  object        
 5   COSMO ID                        156098 non-null  object        
 6   Transaction Type Details        156098 non-null  object        
 7   Sales ID BS                     156098 non-null  object        
 8   MAT_Material(key)               156098 non-null  object        
 9   UNIT SOLD                       156098 non-null  float64       
 10  Discount %                      12921 non-null   float64

##### 3. 마이너스 세일즈, 수량 고객 제외할것 있음하기 >> 없을 것임

In [84]:
df0.groupby(['COSMO ID', 'Last and First Name (Local)', 'Birthdate Year', 'First Purchase Date']).agg({'UNIT SOLD': 'sum', 'MAT_Material(key)': 'nunique', 'NET VALUE': 'sum'})\
.sort_values('NET VALUE', ascending=True).reset_index()

,COSMO ID,Last and First Name (Local),Birthdate Year,First Purchase Date,UNIT SOLD,MAT_Material(key),NET VALUE
0,0015800000wU1SyAAK,장홍자,Not assigned,2006-10-12 00:00:00,2.000,1,180000.000
1,0015800000w4WiYAAU,공경민,Not assigned,2012-03-05 00:00:00,1.000,1,182000.000
2,0015800001N4duSAAR,나영록,Not assigned,2018-06-01 00:00:00,1.000,1,182000.000
3,0014K00000UcUZ8QAN,여운호,Not assigned,2021-12-24 00:00:00,1.000,1,200000.000
4,0014K00000N8U6oQAF,이인자,Not assigned,2021-06-07 00:00:00,1.000,1,200000.000
...,...,...,...,...,...,...,...
86326,0015800000wU3jgAAC,원종남,Not assigned,2009-05-08 00:00:00,5.000,4,1429483000.000
86327,0015800000w4WrLAAU,안지선,1974.000,2012-09-13 00:00:00,25.000,25,1454473550.000
86328,0014K000005NIczQAG,조현아,Not assigned,2020-09-02 00:00:00,15.000,15,1606179500.000
86329,0015800000wU3WxAAK,임주현,Not assigned,2010-03-10 00:00:00,40.000,39,2140686500.000


In [72]:
# df5 = df0.groupby(['COSMO ID', 'Last and First Name (Local)', 'Birthdate Year', 'First Purchase Date']).agg({'UNIT SOLD': 'sum', 'MAT_Material(key)': 'nunique', 'NET VALUE': 'sum'})\
# .sort_values('NET VALUE', ascending=True).reset_index()
# print(len(df5))

In [73]:
# df5[df5['NET VALUE'] < 200000]

In [74]:
# #제품 2개에 20만원 이하 고객 실제 구매제품을 확인하기 어려워 제외함
# novalue = list(df5[(df5['NET VALUE'] < 180000)]['COSMO ID'])
# len(novalue)

In [75]:
# print(df0['NET VALUE'].sum()) #제외전 금액
# print(df0[df0['COSMO ID'].isin(novalue)]['NET VALUE'].sum()) #제외되는 금액

In [76]:
# df0 = df0[~df0['COSMO ID'].isin(novalue)]
# print(df0['NET VALUE'].sum()) #제외후 금액
# df0.reset_index(drop=True, inplace=True)

##### 4. 비회원, 단체구매(SK), 수량 제일 많은(118개) 다이고 의심고객 제외 (고객기준 장바구니와 코호트분석시 제외할것) df0: sales_transaction.xlsx으로 저장후 다음단계에서 조작

In [77]:
# df0 = df0[(df0['COSMO ID'] != '0015800000z8vxWAAQ') & (df0['COSMO ID'] != '0014K00000MU5zUQAT') & (df0['COSMO ID'] != '0014K00000Q0BdTQAV') & (df0['COSMO ID'] != '0015800000wUoGzAAK')]
# df0.reset_index(drop=True, inplace=True)
# print(len(df0))
# print(df0['COSMO ID'].nunique())
# print(df4['NET VALUE'].sum() - df0['NET VALUE'].sum()) #고객 분석에 쓰는 데이터에서 3억 감소
# print(df0['NET VALUE'].sum()) # 비회원은 total sales의 0.5%정도. 

#### 1-3) 제품 database합치기(**df:sales_transaction_basket.xlsx**)

In [78]:
%%time
# 제품 속성 합칠 것
df6 = pd.read_excel('DOS_Sales_2021.11.17.xlsx', sheet_name='ACT')
df6.drop_duplicates(subset=['Sap Code'], keep='last', inplace=True, ignore_index=True)
len(df6)

Wall time: 30.5 s


12869

In [79]:
df6 = df6.iloc[:, 1:-1] #제품속성만 남김
df6.rename(columns={'Sap Code':'MAT_Material(key)', 'Gender':'MAT_Gender'}, inplace=True)
df0 = pd.merge(df0, df6, how='left', on='MAT_Material(key)') # 제품속성 병합
df0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 156098 entries, 0 to 156097
Data columns (total 53 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   TIME-Year                       156098 non-null  int64         
 1   TIME-Month                      156098 non-null  int64         
 2   TIME-Date                       156098 non-null  datetime64[ns]
 3   Purchase: Shop                  156098 non-null  object        
 4   Main Client Advisor             156070 non-null  object        
 5   COSMO ID                        156098 non-null  object        
 6   Transaction Type Details        156098 non-null  object        
 7   Sales ID BS                     156098 non-null  object        
 8   MAT_Material(key)               156098 non-null  object        
 9   UNIT SOLD                       156098 non-null  float64       
 10  Discount %                      12921 non-null   float64

In [80]:
df0 = df0[~df0['Short Description'].isnull()] #제품 맵핑 안된것 확인후 지우기
df0.reset_index(drop=True, inplace=True)
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156097 entries, 0 to 156096
Data columns (total 53 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   TIME-Year                       156097 non-null  int64         
 1   TIME-Month                      156097 non-null  int64         
 2   TIME-Date                       156097 non-null  datetime64[ns]
 3   Purchase: Shop                  156097 non-null  object        
 4   Main Client Advisor             156069 non-null  object        
 5   COSMO ID                        156097 non-null  object        
 6   Transaction Type Details        156097 non-null  object        
 7   Sales ID BS                     156097 non-null  object        
 8   MAT_Material(key)               156097 non-null  object        
 9   UNIT SOLD                       156097 non-null  float64       
 10  Discount %                      12921 non-null   float64

In [81]:
%%time
# SBU 합칠 것
df9 = pd.read_excel('DOS_Sales_2021.11.17.xlsx', sheet_name='SBU')
df9.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   MAT_SBU                 32 non-null     object
 1   MAT_Business Code(key)  32 non-null     object
 2   MAT_Business Code       32 non-null     object
 3   SBU                     32 non-null     object
 4   JWA                     32 non-null     object
 5   MAT_Business Code(num)  32 non-null     int64 
dtypes: int64(1), object(5)
memory usage: 1.6+ KB
Wall time: 13.1 s


In [82]:
df9 = df9[['MAT_SBU', 'MAT_Business Code(key)', 'SBU', 'JWA']]
df0 = pd.merge(df0, df9, how='left', on='MAT_Business Code(key)') # SBU 병합
df0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 156097 entries, 0 to 156096
Data columns (total 56 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   TIME-Year                       156097 non-null  int64         
 1   TIME-Month                      156097 non-null  int64         
 2   TIME-Date                       156097 non-null  datetime64[ns]
 3   Purchase: Shop                  156097 non-null  object        
 4   Main Client Advisor             156069 non-null  object        
 5   COSMO ID                        156097 non-null  object        
 6   Transaction Type Details        156097 non-null  object        
 7   Sales ID BS                     156097 non-null  object        
 8   MAT_Material(key)               156097 non-null  object        
 9   UNIT SOLD                       156097 non-null  float64       
 10  Discount %                      12921 non-null   float64

In [83]:
%%time
df0.to_excel('sales_transaction_basket_cosmo.xlsx', index=False)

Wall time: 2min 25s
